# AskNews

> [AskNews](https://asknews.app) infuses any LLM with the latest global news (or historical news), using a single natural language query. Specifically, AskNews is enriching over 300k articles per day by translating, summarizing, extracting entities, and indexing them into hot and cold vector databases. AskNews puts these vector databases on a low-latency endpoint for you. When you query AskNews, you get back a prompt-optimized string that contains all the most pertinent enrichments (e.g. entities, classifications, translation, summarization). This means that you do not need to manage your own news RAG, and you do not need to worry about how to properly convey news information in a condensed way to your LLM.
> AskNews is also committed to transparency, which is why our coverage is monitored and diversified across hundreds of countries, 13 languages, and 50 thousand sources. If you'd like to track our source coverage, you can visit our [transparency dashboard](https://asknews.app/en/transparency).

## Setup

The integration lives in the `langchain-community` package. We also need to install the `asknews` package itself.

```bash
pip install -U langchain-community asknews
```

We also need to set our AskNews API credentials, which can be obtained at the [AskNews console](https://my.asknews.app).

In [2]:
import getpass
import os

os.environ["ASKNEWS_CLIENT_ID"] = getpass.getpass()
os.environ["ASKNEWS_CLIENT_SECRET"] = getpass.getpass()

It's also helpful (but not needed) to set up [LangSmith](https://smith.langchain.com/) for best-in-class observability

In [ ]:
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

## Usage

Here we show how to use the tool individually.

In [3]:
from langchain_community.tools.asknews import AskNewsSearch

tool = AskNewsSearch()

In [4]:
tool.invoke({"query": "Effect of fed policy on tech sector"})

"\n<doc>\n[1]:\ntitle: Wells Fargo sees increasing risk that the Fed will need to hold rates at current levels\nsummary: Wells Fargo has increased its assessment of the risk that the Federal Reserve will need to keep interest rates steady due to high inflation, which could push the 2-year Treasury yield to 5%. The firm believes that the Fed will attempt to cut rates twice before the end of the year, but the probability of holding rates steady is rising. As a result, Wells Fargo is taking a defensive approach in its positioning, recommending that fixed-income investors favor short-term fixed income and be neutral on intermediate and long-term investments. The firm also expects the yield curve to continue to flatten, and notes that events such as the Fed removing rate cut expectations, inflation increasing, or the US fiscal situation deteriorating could push 10-year Treasury yields towards 5%.\nsource: Seeking Alpha\npublished: May 12 2024 13:30\nOrganization: Federal Reserve, Fed, Wells

## Chaining
We show here how to use it as part of an [agent](/docs/modules/agents). We use the OpenAI Functions Agent, so we will need to setup and install the required dependencies for that. We will also use [LangSmith Hub](https://smith.langchain.com/hub) to pull the prompt from, so we will need to install that.

```bash
pip install -U langchain-openai langchainhub
```

In [6]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

In [7]:
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain_openai import ChatOpenAI

instructions = """You are an assistant."""
base_prompt = hub.pull("langchain-ai/openai-functions-template")
prompt = base_prompt.partial(instructions=instructions)
llm = ChatOpenAI(temperature=0)
asknews_tool = AskNewsSearch()
tools = [asknews_tool]
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
)

In [9]:
agent_executor.invoke({"input": "How is the tech sector being affected by fed policy?"})



> Entering new AgentExecutor chain...

Invoking: `asknews_search` with `{'query': 'tech sector impact of fed policy'}`



<doc>
[1]:
title: US Stock Market Declines Amid High Interest Rates
summary: The US stock market has experienced a significant decline in recent days, with the S&P 500 index falling 9.94 points or 0.2% to 4320.06. The decline was attributed to interest rates, which are expected to remain high for a longer period. The yield on 10-year Treasury notes rose to 4.44%, the highest level since 2007, which has a negative impact on stock prices. The high interest rates have also affected the technology sector, with companies such as Intel and Microsoft experiencing declines. The auto sector is also experiencing fluctuations, with General Motors and Ford experiencing declines. The labor market is also facing challenges, with workers demanding higher wages and benefits, which could lead to increased inflation. The Federal Reserve is expected to keep interest rates high for a

{'input': 'How is the tech sector being affected by fed policy?',
 'output': "The tech sector is being affected by Federal Reserve policy, particularly in relation to interest rates and market conditions. The recent increase in interest rates has had a negative impact on the stock market, with the S&P 500 index falling and technology companies like Intel and Microsoft experiencing declines. The Federal Reserve's decision to maintain high interest rates for a longer period is contributing to market fluctuations and challenges in various sectors, including technology. Additionally, experts warn of a potential economic downturn in the US, which could further impact the tech sector and lead to job losses and closures in companies without government support. The stock market is also showing signs of volatility and speculative behavior, reminiscent of past market crises like the dotcom bubble."}